In [1]:
from netCDF4 import Dataset
import numpy as np

# Reading netCDF file

In [43]:
data_1975 = Dataset("1961.nc")
data_2015 = Dataset("2015.nc")

# Displaying names of the variables

In [44]:
data_1975.variables.keys()

dict_keys(['lon', 'lat', 'time', 'tave', 'rstn'])

In [4]:
data_2015.variables.keys()

dict_keys(['lon', 'lat', 'time', 'tave', 'rstn'])

# Accessing the variables

In [59]:
lon = data_1975.variables['lon']
lat = data_1975.variables['lat']
temp_1975 = data_1975.variables['tave']
temp_2015 = data_2015.variables['tave']

In [60]:
temp_1975

<class 'netCDF4._netCDF4.Variable'>
float32 tave(time, lat, lon)
    _FillValue: -99.9
    units: degrees centigrade
    long_name:  daily mean temperature analysis interpolated onto 0.25deg grids
unlimited dimensions: time
current shape = (365, 280, 360)
filling on

# Accessing the data from the variables

In [79]:
lon_list = []
for each_lon in lon:
    lon_list.append(each_lon.data.tolist())
lat_list = []
for each_lat in lat:
    lat_list.append(each_lat.data.tolist())

# Getting a datafrom containing location and two-year temperature on 06/01

In [11]:
import pandas as pd

In [63]:
dt1 = np.arange(0,lon.size)
dt2 = np.arange(0,lat.size)

In [160]:
longitude = []
latitude = []
temps_1975 = []
temps_2015 = []

for lon in dt1: 
    for lat in dt2: 
        actual_lon = lon_list[lon]
        actual_lat = lat_list[lat]
        temp_day_1975 = temp_1975[151, lat, lon].data.tolist()
        temp_day_2015 = temp_2015[151, lat, lon].data.tolist()
        longitude.append(actual_lon)
        latitude.append(actual_lat)
        temps_1975.append(temp_day_1975)
        temps_2015.append(temp_day_2015)

In [161]:
df = pd.DataFrame()
df['longitude']=longitude
df['latitude']=latitude
df['temp_1975']=temps_1975
df['temp_2015']=temps_2015

In [162]:
df['diff'] = df['temp_2015'] - df['temp_1975']

In [163]:
df.tail()

,longitude,latitude,temp_1975,temp_2015,diff
100795,149.875,53.875,0.0,0.0,0.0
100796,149.875,54.125,0.0,0.0,0.0
100797,149.875,54.375,0.0,0.0,0.0
100798,149.875,54.625,0.0,0.0,0.0
100799,149.875,54.875,0.0,0.0,0.0


# Exporting as geojson file and mapping

In [118]:
import geopandas as gpd
import altair as alt

In [164]:
dfGeo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [165]:
dfGeo.sample(10)

,longitude,latitude,temp_1975,temp_2015,diff,geometry
56036,110.125,-5.875,0.000000,0.000000,0.000000,POINT (110.12500 -5.87500)
40757,96.375,24.375,27.512358,27.028601,-0.483757,POINT (96.37500 24.37500)
13695,72.125,48.875,17.309090,16.382975,-0.926115,POINT (72.12500 48.87500)
5440,64.875,15.125,0.000000,0.000000,0.000000,POINT (64.87500 15.12500)
61702,115.125,10.625,0.000000,0.000000,0.000000,POINT (115.12500 10.62500)
31545,88.125,31.375,7.237032,7.170112,-0.066920,POINT (88.12500 31.37500)
42641,98.125,5.375,0.000000,0.000000,0.000000,POINT (98.12500 5.37500)
95271,145.125,2.875,0.000000,0.000000,0.000000,POINT (145.12500 2.87500)
15237,73.625,14.375,0.000000,0.000000,0.000000,POINT (73.62500 14.37500)
67666,120.375,31.625,19.482504,25.156189,5.673685,POINT (120.37500 31.62500)


In [166]:
dfGeo_small = dfGeo[(dfGeo.temp_1975 != 0) & (dfGeo.temp_2015 != 0)]

In [167]:
dfGeo_small.shape

(46489, 6)

In [168]:
dfGeo_small.describe()

,longitude,latitude,temp_1975,temp_2015,diff
count,46489.000000,46489.000000,46489.000000,46489.000000,46489.000000
mean,98.538980,33.243781,19.733175,19.903743,0.170568
std,21.430206,16.127345,9.251933,9.214181,3.820882
min,60.125000,-14.875000,-6.283364,-6.247631,-11.253923
25%,80.125000,24.625000,13.223133,12.147732,-2.124088
50%,99.125000,36.125000,21.069628,20.993206,0.394598
75%,114.625000,46.125000,26.841795,27.668570,2.618221
max,149.875000,54.875000,39.294197,38.972389,10.541774


In [169]:
dfGeo_small.head()

,longitude,latitude,temp_1975,temp_2015,diff,geometry
161,60.125,25.375,29.385006,35.375866,5.990860,POINT (60.12500 25.37500)
162,60.125,25.625,29.387630,35.420658,6.033028,POINT (60.12500 25.62500)
163,60.125,25.875,29.445625,35.530720,6.085094,POINT (60.12500 25.87500)
164,60.125,26.125,29.583069,35.664291,6.081223,POINT (60.12500 26.12500)
165,60.125,26.375,28.191296,34.304989,6.113693,POINT (60.12500 26.37500)


In [147]:
alt.Chart(dfGeo.sample(1000)).mark_geoshape().encode(
    color=alt.Color('diff:Q', title='temperature difference', scale=alt.Scale(scheme='inferno')),
    tooltip=['temp_1975:Q','temp_2015:Q','diff'],
).properties(
    title='Asian temperature change from 1975 to 2015'
).configure_view(
    strokeWidth=0
)

alt.Chart(...)

In [170]:
dfGeo.to_file('asianTempAll.geojson', driver='GeoJSON')
dfGeo_small.to_file('asianTemp.geojson', driver='GeoJSON')